In [32]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

## Dictionary features

In [33]:
personalities = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')

In [34]:
personalities.shape

(1039, 186)

In [35]:
personalities.head()

,Unnamed: 0,file_name,formatted_text,X,sentences,items,chars,allTokens,wordTokens,skip,...,extraversion_m,agreeabeness_m,conscientiousness_m,neuroticism_m,openness_m,extraversion_ober_2,agreeableness_ober_2,conscientiousness_ober_2,neuroticism_ober_2,openness_ober_2
0,1,100,ajudando porque a zuzu é um amor e tem a voz f...,1,230,2032,9649,968,609,0.0089,...,0,0,0,1,1,0,0,0,1,1
1,2,1000,Chega de saudade ! A festa brasileira mais b...,2,240,2538,12020,965,870,0.0004,...,0,0,1,1,0,0,-1,-1,1,0
2,3,1001,Se tem um filme em todas as galáxias existen...,3,247,4245,19933,1503,1353,0,...,0,1,0,0,0,-1,1,-1,-1,-1
3,4,1002,"Sobre ter vizinhas prendadas e fofas , que t...",4,206,3982,19492,1328,1151,0.0005,...,1,0,1,1,0,1,0,1,-1,-1
4,5,1003,Nostalgia Agora eu entendi Só rindo da pia...,5,136,805,4448,477,401,0,...,1,1,0,1,1,1,1,-1,-1,1


In [36]:
personalities = personalities.convert_objects(convert_numeric=True)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [37]:
personalities.agreeabeness_m.value_counts()

1    542
0    497
Name: agreeabeness_m, dtype: int64

In [38]:
train_data, test_data = train_test_split(personalities, test_size=0.3, random_state=42)

In [39]:
pre_proc_attr = ['skip', 'compound', 'hashtags', 'links', 'punct', 'questions', 'exclam', 'numbers', 'upcase', 'lowcase', 'firstup', 'pt.lexicon', 'added', 'verb.pro', 'names', 'en.lexicon', 'rewrite', 'mispell', 'foreign', 'laugh', 'emph', 'echars', 'unkn']

In [40]:
grammar_attr = ['m', 'f', 's', 'p', 'aument', 'dimin', 'superlat', 'N', 'A', 'PREP', 'CONJ', 'ADV', 'PREFIX', 'SIGLA', 'ABREV', 'INTERJ', 'DET', 'def', 'indef', 'NUM', 'numC', 'numO', 'numM', 'numF', 'PRO', 'proDem', 'proIndef', 'proRel', 'proInterr', 'proTrat', 'proPoss', 'proPess', 'acusativa', 'dativa', 'nominativa', 'obliqua', 'reflexa', 'p1', 'p2', 'p3', 'V', 'VW', 'VG', 'VK', 'VP', 'VI', 'VJ', 'VF', 'VQ', 'VS', 'VT', 'VU', 'VY', 'VC', 'V1s', 'V2s', 'V3s', 'V1p', 'V2p', 'V3p']

In [41]:
liwc_attr = ['X1funct' ,'X2pronoun' ,'X3ppron','X4i','X5we','X6you','X7shehe','X8they','X9ipron','X10article','X11verb','X12auxverb','X13past','X14present','X15future','X16adverb','X17preps','X18conj','X19negate','X20quant','X21number','X22swear','X23social','X24family','X25friend','X26humans','X27affect','X28posemo','X29negemo','X30anx','X31anger','X32sad','X33cogmech','X34insight','X35cause','X36discrep','X37tentat','X38certain','X39inhib','X40incl','X41excl','X42percept','X43see','X44hear','X45feel','X46bio','X47body','X48health','X49sexual','X50ingest','X51relativ','X52motion','X53space','X54time','X55work','X56achieve','X57leisure','X58home','X59money','X60relig','X61death','X62assent','X63nonfl', 'X64filler']

In [42]:
all_attr = train_data.columns[7:157]

In [43]:
def remove_nan(atributes):
    train_attributes = np.array(train_data[atributes])
    test_attributes = np.array(test_data[atributes])
    train_attributes[np.isnan(train_attributes)] = 0
    test_attributes[np.isnan(test_attributes)] = 0
    return (train_attributes, test_attributes)

In [44]:
train, test = remove_nan(liwc_attr)

In [119]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [3,5,10],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced", "balanced_subsample"],
        "random_state": [42]
    },
    scoring="f1_weighted",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

## Extraversion

## Agreableness

In [120]:
%%time
model_trainer.fit(train, train_data['agreeableness_ober_2'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


CPU times: user 279 ms, sys: 176 ms, total: 455 ms
Wall time: 695 ms


In [121]:
model

RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='log2',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [122]:
print train.shape
print train_data['agreeableness_ober_2'].value_counts()

(727, 64)
-1    501
 1    126
 0    100
Name: agreeableness_ober_2, dtype: int64


In [123]:
yp = model.predict(test)
yt = test_data['agreeableness_ober_2']

In [127]:
yt.value_counts()

-1    195
 1     70
 0     47
Name: agreeableness_ober_2, dtype: int64

In [124]:
pd.crosstab(yt, yp, rownames=['Actual'], colnames=['Predicted'])



Predicted,-1,0,1
Actual,,,
-1,161,14,20
0,40,4,3
1,59,2,9


In [125]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

         -1       0.62      0.83      0.71       195
          0       0.20      0.09      0.12        47
          1       0.28      0.13      0.18        70

avg / total       0.48      0.56      0.50       312



In [126]:
accuracy_score(yt, yp)

0.55769230769230771